In [2]:
from qiskit import QuantumCircuit,transpile,QuantumRegister,ClassicalRegister
from qiskit_aer import Aer 
from qiskit.visualization import *
from qiskit_ibm_runtime import QiskitRuntimeService

In [3]:
service = QiskitRuntimeService(channel='ibm_quantum')

In [4]:
q = QuantumRegister(3,'q')
c0 = ClassicalRegister(1,'c0')
c1 = ClassicalRegister(1,'c1')
c2 = ClassicalRegister(1,'c2')
circuit = QuantumCircuit(q,c0,c1,c2)

In [5]:
print(circuit)

      
 q_0: 
      
 q_1: 
      
 q_2: 
      
c0: 1/
      
c1: 1/
      
c2: 1/
      


In [6]:
circuit.initialize([0,1],0)
circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

In [7]:
print(circuit)

      ┌─────────────────┐ ░ 
 q_0: ┤ Initialize(0,1) ├─░─
      └─────────────────┘ ░ 
 q_1: ────────────────────░─
                          ░ 
 q_2: ────────────────────░─
                          ░ 
c0: 1/══════════════════════
                            
c1: 1/══════════════════════
                            
c2: 1/══════════════════════
                            


In [8]:
circuit.h(q[1])
circuit.cx(q[1],q[2])
circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

In [10]:
print(circuit)

      ┌─────────────────┐ ░            ░ 
 q_0: ┤ Initialize(0,1) ├─░────────────░─
      └─────────────────┘ ░ ┌───┐      ░ 
 q_1: ────────────────────░─┤ H ├──■───░─
                          ░ └───┘┌─┴─┐ ░ 
 q_2: ────────────────────░──────┤ X ├─░─
                          ░      └───┘ ░ 
c0: 1/═══════════════════════════════════
                                         
c1: 1/═══════════════════════════════════
                                         
c2: 1/═══════════════════════════════════
                                         


In [12]:
circuit.cx(q[0],q[1])
circuit.h(q[0])
circuit.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=3, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(3, 'q'), 0), Qubit(QuantumRegister(3, 'q'), 1), Qubit(QuantumRegister(3, 'q'), 2)), clbits=())

In [13]:
print(circuit)

      ┌─────────────────┐ ░            ░      ┌───┐ ░ 
 q_0: ┤ Initialize(0,1) ├─░────────────░───■──┤ H ├─░─
      └─────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ 
 q_1: ────────────────────░─┤ H ├──■───░─┤ X ├──────░─
                          ░ └───┘┌─┴─┐ ░ └───┘      ░ 
 q_2: ────────────────────░──────┤ X ├─░────────────░─
                          ░      └───┘ ░            ░ 
c0: 1/════════════════════════════════════════════════
                                                      
c1: 1/════════════════════════════════════════════════
                                                      
c2: 1/════════════════════════════════════════════════
                                                      


In [14]:
circuit.measure(q[0],c0[0])
circuit.measure(q[1],c1[0])

In [15]:
circuit.x(q[2]).c_if(c1,1)
circuit.z(q[2]).c_if(c0,1)

In [16]:
circuit.measure(q[2],c2[0])

In [17]:
print(circuit)

      ┌─────────────────┐ ░            ░      ┌───┐ ░ ┌─┐                    
 q_0: ┤ Initialize(0,1) ├─░────────────░───■──┤ H ├─░─┤M├────────────────────
      └─────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐                 
 q_1: ────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─────────────────
                          ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ┌───┐  ┌───┐ ┌─┐
 q_2: ────────────────────░──────┤ X ├─░────────────░──╫──╫──┤ X ├──┤ Z ├─┤M├
                          ░      └───┘ ░            ░  ║  ║  └─╥─┘  └─╥─┘ └╥┘
                                                       ║  ║    ║   ┌──╨──┐ ║ 
c0: 1/═════════════════════════════════════════════════╩══╬════╬═══╡ 0x1 ╞═╬═
                                                       0  ║ ┌──╨──┐└─────┘ ║ 
c1: 1/════════════════════════════════════════════════════╩═╡ 0x1 ╞════════╬═
                                                          0 └─────┘        ║ 
c2: 1/══════════════════════════════════════════════════════════

In [18]:
simulator = Aer.get_backend('qasm_simulator')
result = simulator.run(circuit,shots=1).result()
counts = result.get_counts(circuit)

In [19]:
counts

{'1 1 1': 1}